# LangChain: Function Calling

In [1]:
# Importando minha Api Key da OpenAI que se encontra na minha variável de ambiente
import os
from dotenv import load_dotenv
import openai
import json

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


## Function Calling

Trata da capacidade dos modelos de poderem utilizar fontes externas como APIs para gerarem respostas mais precisas aos usuários, evitando problemas de alucinação do modelo.

In [2]:
# Função básica de chamada de modelo e obtenção de respostas. Vale ressaltar que o modelo utilizado deve
# Suportar as Functions Callings.
def get_completion (messages, model="gpt-3.5-turbo-1106", temperature=0, tools=None, tool_choice=None):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        tools=tools,
        tool_choice=tool_choice
    )
    return response.choices[0].message

### Definindo uma Dummy Function

Função fictícia para testes

In [3]:
# Essa função servirá como ferramenta para o modelo
def get_current_weather(location, unit="celsius"):
    """Pega a temperatura atual em uma determinada localização"""
    weather = {
        "location": location,
        "temperature": "30",
        "unit": unit,
    }
    return json.dumps(weather)

### Definindo a função em uma variável Tools que será passada para o modelo

In [4]:
tools = [
    {
        "type":"function",
        "function": {
            "name":"get_current_weather",
            "description":"Pega a temperatura atual em uma determinada localização",
            "parameters": {
                "type":"object",
                "properties":{
                    "location": {
                        "type":"string",
                        "description":"A cidade e o estado, e.g. Goiana, PE"
                    },
                    "unit":{
                        "type":"string",
                        "enum":["celsius","fahrenheit"]
                    }
                },
                "required":["location"]
            }
        }
    }

]

In [5]:
# Definindo o prompt

messages = [
    {"role":"user",
     "content":"Qual a temperatura em Londrina por favor?"}
]

In [6]:
# Aqui é possível verificar como o modelo escolheu os parâmetros da função com base no que escrevi
# no Prompt
response = get_completion(messages, tools=tools)
print(response)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zo7uqFAb3ahj0jWUOnxd8jAV', function=Function(arguments='{"location":"Londrina","unit":"celsius"}', name='get_current_weather'), type='function')])


In [7]:
# Extraindo os argumentos da função
args = json.loads(response.tool_calls[0].function.arguments)

In [8]:
args

{'location': 'Londrina', 'unit': 'celsius'}

In [9]:
get_current_weather(**args)

'{"location": "Londrina", "temperature": "30", "unit": "celsius"}'

### Controlando o comportamento do Function Calling

Uma forma de controlar o comportamento para evitar que a função seja chamada desnecessariamente.

In [10]:
messages = [
    {"role":"user",
     "content":"Oi, oi!"}
]

In [11]:
get_completion(messages, tools=tools)

ChatCompletionMessage(content='Olá! Como posso ajudar você hoje?', refusal=None, role='assistant', function_call=None, tool_calls=None)

Por padrão, o modelo decide por si só quando chamar uma tool, mas isso pode facilmente ser configurado utilizando a ferramenta de ``tool_choice`` que por padrão vem configurada como ``tool_choice="auto"``.    

In [12]:
get_completion(messages, tools=tools, tool_choice="auto")

ChatCompletionMessage(content='Olá! Como posso ajudar você hoje?', refusal=None, role='assistant', function_call=None, tool_calls=None)

``tool_choice="none"`` não permite que o modelo use nenhuma função disponibilizada para ele.

In [13]:
messages = [
    {"role":"user",
     "content":"Qual a temperatura em Londres por favor?"}
]

In [14]:
get_completion(messages, tools=tools, tool_choice="none")

ChatCompletionMessage(content='Por favor, aguarde um momento enquanto verifico a temperatura atual em Londres para você.', refusal=None, role='assistant', function_call=None, tool_calls=None)

Acima é possível observar que o modelo foi forçado a não utilizar a função passada para ele anteriormente.

Também é possível forçar o modelo a usar uma função específica ao seu querer. Por exemplo, em uma variável tools que tem muitas funções, em determinados prompts você pode passar uma função específica para ajudar o modelo a não precisar perder tempo analizando todas as outras.

In [15]:
messages = [
    {"role":"user",
     "content":"Qual a temperatura em Londres por favor?"}
]

In [16]:
get_completion(messages, tools=tools, tool_choice={"type":"function", "function":{"name":"get_current_weather"}})

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ZcV0n78N4ghqbo8skKvuqHK3', function=Function(arguments='{"location":"London","unit":"celsius"}', name='get_current_weather'), type='function')])